In [ ]:
from telegram import Bot
import nest_asyncio
import asyncio
import os
# Set the environment variable
os.environ['BOT_TOKEN'] = '[Token]'
# Verify that it's set
print(f"BOT_TOKEN: {os.environ.get('BOT_TOKEN')}")

In [ ]:
import logging
from telegram import (ReplyKeyboardMarkup, ReplyKeyboardRemove, Update, InlineKeyboardButton, InlineKeyboardMarkup)
from telegram.ext import (Application, CallbackQueryHandler, CommandHandler, ContextTypes, ConversationHandler, MessageHandler, filters)

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
HOUSE_TYPE, LOCATION, RENTING_PRICE_DECISION, RENTING_PRICE, PHOTO, SUMMARY = range(6)

In [ ]:
def main() -> None:
    """Run the bot."""
    application = Application.builder().token("YOUR TOKEN HERE").build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            HOUSE_TYPE: [MessageHandler(filters.TEXT & ~filters.COMMAND, house_type)],
            LOCATION: [CallbackQueryHandler(location)],
            RENTING_PRICE_DECISION: [CallbackQueryHandler(renting_price_decision)],
            RENTING_PRICE: [MessageHandler(filters.TEXT & ~filters.COMMAND, renting_price)],
            PHOTO: [
                MessageHandler(filters.PHOTO, photo),
                CommandHandler('skip', skip_photo)
            ],
            SUMMARY: [MessageHandler(filters.ALL, summary)]
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    application.add_handler(conv_handler)

    # Handle the case when a user sends /start but they're not in a conversation
    application.add_handler(CommandHandler('start', start))

    application.run_polling()

In [ ]:
import uvloop
import asyncio
asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Starts the conversation and asks the user about their house listing type."""
    reply_keyboard = [['HDB', 'Terrace', 'GCB', 'Condo']]
    
    image_path = 'Screenshot 2024-09-26 at 8.19.01 PM.png'
    with open(image_path, 'rb') as image_file:
        
        await context.bot.send_photo(chat_id = '689244075', photo = image_file)

    await update.message.reply_text(
        '<b>Hello and Welcome to the House Sales Listing Bot!\n'
        '\n'
        "We're proud to be one of Singapore's leading property agencies, recognized with top accolades for the highest number of transactions in 2023. \n"
        
        "To help connect you with one of our expert agents, let's gather some details about the property you're selling. \n"
        
        'First, could you tell us the type of property you are listing?</b>',
        
        parse_mode='HTML',
        reply_markup=ReplyKeyboardMarkup(
            reply_keyboard, one_time_keyboard=True, resize_keyboard=True
        ),
    )

    return HOUSE_TYPE

            
            
            
            

async def house_type(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Stores the user's house details"""
    user = update.message.from_user
    context.user_data['house_type'] = update.message.text
    house = {"HDB": "🏡", "Condo": "🏢", "Terrace": "🏠︎", "GCB": "🏬"}
    logger.info('House type of %s: %s', user.first_name, update.message.text)
    await update.message.reply_text(
        f'<b>You selected {update.message.text} House {house[update.message.text]}, Fabulous!!! \n'
        f'Where is the location of your house?</b>',
        parse_mode='HTML',
        reply_markup=ReplyKeyboardRemove(),
    )

    # Define inline buttons for House color selection
    keyboard = [
        [InlineKeyboardButton('Central', callback_data='Central')],
        [InlineKeyboardButton('East Region', callback_data='East Region')],
        [InlineKeyboardButton('North Region', callback_data='North Region')],
        [InlineKeyboardButton('West Region', callback_data='West Region')],
        [InlineKeyboardButton('North-East Region', callback_data='North-East Region')],
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await update.message.reply_text('<b>Please choose:</b>', parse_mode='HTML', reply_markup=reply_markup)

    return LOCATION


async def location(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Stores the user's House Location Selection."""
    query = update.callback_query
    await query.answer()
    context.user_data['house_location'] = query.data
    await query.edit_message_text(
        text=f'<b>You selected {query.data}.\n'
             f'Would you like to fill in the price you will like to rent your house at?</b>',
        parse_mode='HTML'
    )

    # Define inline buttons for renting price decision
    keyboard = [
        [InlineKeyboardButton('Fill in rental price', callback_data='Fill')],
        [InlineKeyboardButton('Skip', callback_data='Skip')],
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    await query.message.reply_text('<b>Choose an option:</b>', parse_mode='HTML', reply_markup=reply_markup)

    return RENTING_PRICE_DECISION


async def renting_price_decision(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Asks the user to fill in the rental price or skip."""
    query = update.callback_query
    await query.answer()
    decision = query.data

    if decision == 'Fill':
        await query.edit_message_text(text='<b>Please type in the rental price (e.g., 5000):</b>', parse_mode='HTML')
        return RENTING_PRICE
    else:
        await query.edit_message_text(text='<b>Renting_Price step skipped.</b>', parse_mode='HTML')
        return await skip_renting_price(update, context)


async def renting_price(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Stores the renting price."""
    context.user_data['renting_price'] = update.message.text
    await update.message.reply_text('<b>Renting Price noted.\n'
                                    'Please upload a photo of your house 📷, or send /skip.</b>',
                                    parse_mode='HTML')
    return PHOTO


async def skip_renting_price(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Skips the rental price input."""
    context.user_data['rental_price'] = 'Not provided'

    text = '<b>Please upload a photo of your house 📷, or send /skip.</b>'

    # Determine the correct way to send a reply based on the update type
    if update.callback_query:
        # If called from a callback query, use the callback_query's message
        chat_id = update.callback_query.message.chat_id
        await context.bot.send_message(chat_id=chat_id, text=text, parse_mode='HTML')
        # Optionally, you might want to acknowledge the callback query
        await update.callback_query.answer()
    elif update.message:
        # If called from a direct message
        await update.message.reply_text(text)
    else:
        # Handle other cases or log an error/warning
        logger.warning('skip_renting_price was called without a message or callback_query context.')

    return PHOTO


async def photo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Stores the photo."""
    photo_file = await update.message.photo[-1].get_file()
    # Correctly store the file_id of the uploaded photo for later use
    context.user_data['House_photo'] = photo_file.file_id  # Preserve this line

    # Inform user and transition to summary
    await update.message.reply_text('<b>Photo uploaded successfully.\n'
                                    'Let\'s summarize your selections.</b>',
                                    parse_mode='HTML'
    )
    await summary(update, context)  # Proceed to summary


async def skip_photo(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Skips the photo upload."""
    await update.message.reply_text('<b>No photo uploaded.\n'
                                    'Let\'s summarize your selections.</b>',
                                    parse_mode='HTML')
    await summary(update, context)


async def summary(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Summarizes the user's selections and ends the conversation, including the uploaded image."""
    selections = context.user_data
    # Construct the summary text
    summary_text = (f"<b>Here's what you told me about your house rental:\n</b>"
                    f"<b>House Type:</b> {selections.get('house_type')}\n"
                    f"<b>Location:</b> {selections.get('location')}\n"
                    f"<b>Renting Price:</b> {selections.get('renting_price')}\n"
                    f"<b>Photo:</b> {'Uploaded' if 'house_photo' in selections else 'Not provided'}")

    chat_id = update.effective_chat.id

    # If a photo was uploaded, send it back with the summary as the caption
    if 'house_photo' in selections and selections['house_photo'] != 'Not provided':
        await context.bot.send_photo(chat_id=chat_id, photo=selections['house_photo'], caption=summary_text, parse_mode='HTML')
    else:
        # If no photo was uploaded, just send the summary text
        await context.bot.send_message(chat_id=chat_id, text=summary_text, parse_mode='HTML')

    return ConversationHandler.END

async def cancel(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    """Cancels and ends the conversation."""
    await update.message.reply_text('Bye! Hope to talk to you again soon.', reply_markup=ReplyKeyboardRemove())
    return ConversationHandler.END

#async def unknown(update: Update, context: ContextTypes.DEFAULT_TYPE):
#    await update.message.reply_text('What you talking about?.', reply_markup=ReplyKeyboardRemove())
#    return ConversationHandler.END
    #await context.bot.send_message(chat_id=update.effective_chat.id, text="Sorry, I didn't understand that command.")
    #return ConversationHandler.END
    
async def unknown(update: Update, context: ContextTypes.DEFAULT_TYPE) -> int:
    # Provide feedback for unrecognized input and ask the user to try again
    await update.message.reply_text(
        'Sorry, I didn’t understand that. Please select from the available options or type a valid response.',
        reply_markup=ReplyKeyboardRemove()
    )
    
    # You can return the current state, e.g., HOUSE_TYPE or any other current state
    return HOUSE_TYPE


async def main() -> None:
    """Run the bot."""
    # Use environment variable for the bot token
    BOT_TOKEN = os.getenv('BOT_TOKEN')
    if not BOT_TOKEN:
        print("Error: BOT_TOKEN environment variable not set.")
        return

    application = Application.builder().token(BOT_TOKEN).build()

    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            HOUSE_TYPE: [MessageHandler(filters.TEXT & ~filters.COMMAND, house_type)],
            LOCATION: [CallbackQueryHandler(location)],
            RENTING_PRICE_DECISION: [CallbackQueryHandler(renting_price_decision)],
            RENTING_PRICE: [MessageHandler(filters.TEXT & ~filters.COMMAND, renting_price)],
            PHOTO: [
                MessageHandler(filters.PHOTO, photo),
                CommandHandler('skip', skip_photo)
            ],
            SUMMARY: [MessageHandler(filters.ALL, summary)]
        },
        fallbacks=[CommandHandler('cancel', cancel)],
    )

    application.add_handler(conv_handler)

    # Start the application
    await application.initialize()
    await application.start()
    print("Bot is running... Press Ctrl+C to stop.")
    await application.updater.start_polling()
    # Keep the bot running until manually stopped
    await asyncio.Event().wait()

# For environments that support top-level await (e.g., Jupyter Notebook)
await main()